## Grid Manipulations (merge, split, refine, transform)
### Notes
Most grid transformations such as `merge` and `transpose` return a new object, allowing consecutive operations to be chained together.
Optionally, you can pass `inplace=True` to the call signature to modify the existing object and return `None`.
Both approaches are demonstrated below.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas

import pygridgen as pgg
import pygridtools as pgt

import seaborn
clear_bkgd = {'axes.facecolor':'none', 'figure.facecolor':'none'}
palette = seaborn.color_palette(palette='deep')
seaborn.set(style='ticks', context='notebook', rc=clear_bkgd)

### Basic merging operations
The function below create our 3 test model grids moving counter-clockwise in the figure shown two cells down.

In [ ]:
def make_test_grids():
    domain1 = pandas.DataFrame({'x': [2, 5, 5, 2], 'y': [6, 6, 4, 4], 'beta': [1, 1, 1, 1]})
    domain2 = pandas.DataFrame({'x': [6, 11, 11, 5], 'y': [5, 5, 3, 3], 'beta': [1, 1, 1, 1]})
    domain3 = pandas.DataFrame({'x': [7, 9, 9, 7], 'y': [2, 2, 0, 0], 'beta': [1, 1, 1, 1]})

    grid1 = pgt.makeGrid(domain=domain1, nx=6, ny=5, rawgrid=False)
    grid2 = pgt.makeGrid(domain=domain2, nx=8, ny=7, rawgrid=False)
    grid3 = pgt.makeGrid(domain=domain3, nx=4, ny=10, rawgrid=False)
    
    return grid1, grid2, grid3


Display positions of grids relative to each other

In [ ]:
grid1, grid2, grid3 = make_test_grids()

fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid1.plotCells(ax=ax, cell_kws=dict(facecolor=palette[0], alpha=0.75))
fig = grid2.plotCells(ax=ax, cell_kws=dict(facecolor=palette[1], alpha=0.75))
fig = grid3.plotCells(ax=ax, cell_kws=dict(facecolor=palette[2], alpha=0.75))

#### Merge grids 1 and 2 together, horizontally
By default, the bottom rows are aligned and the cell mask is not updated. We do that manually for now.

In [ ]:
grid1.merge(grid2, how='horiz', inplace=True)
grid1.cell_mask = np.ma.masked_invalid(grid1.xc).mask
fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid1.plotCells(ax=ax, cell_kws=dict(facecolor=palette[3]))

#### Use the shift parameter to center grid 2
Use `shift=-1` since we're sliding grid 2's i-j indexes downward relative to grid 1

In [ ]:
grid1, grid2, grid3 = make_test_grids()
grid1.merge(grid2, how='horiz', shift=-1, inplace=True)
grid1.cell_mask = np.ma.masked_invalid(grid1.xc).mask
fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid1.plotCells(ax=ax, cell_kws=dict(facecolor=palette[4], alpha=0.75))

#### Vertically merge grid 2 and grid 3
Notice that by default, the grids are left-aligned and the *bottom* of grid 3 ties into the *top* of grid 2

In [ ]:
grid1, grid2, grid3 = make_test_grids()
grid2.merge(grid3, how='vert', shift=2, inplace=True)
grid2.cell_mask = np.ma.masked_invalid(grid2.xc).mask
fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid2.plotCells(ax=ax, cell_kws=dict(facecolor='firebrick'))

#### Try again, switching the order of the grids
Notice the change in sign of the `shift` parameter.

In [ ]:
grid1, grid2, grid3 = make_test_grids()
grid3.merge(grid2, how='vert', shift=-2, inplace=True)
grid3.cell_mask = np.ma.masked_invalid(grid3.xc).mask
fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid3.plotCells(ax=ax, cell_kws=dict(facecolor=palette[5], alpha=0.75))

#### Alternatively, you can switch the arguments and use `where='-'` to indicate that the "other" grid is below the first.
And the sign of the `shift` parameter returns to its original value.

In [ ]:
grid1, grid2, grid3 = make_test_grids()
grid2.merge(grid3, how='vert', where='-', shift=2, inplace=True)
grid2.cell_mask = np.ma.masked_invalid(grid2.xc).mask
fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid2.plotCells(ax=ax, cell_kws=dict(facecolor=palette[0], alpha=0.75))

#### Now merge all three in a single chained operation (`inplace=False`).

In [ ]:
grid1, grid2, grid3 = make_test_grids()
grid = (
    grid2.merge(grid3, how='vert', where='-', shift=2)
         .merge(grid1, how='horiz', where='-', shift=11) 
)

grid.cell_mask = np.ma.masked_invalid(grid.xc).mask
fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid.plotCells(ax=ax, cell_kws=dict(facecolor=palette[1], alpha=0.75))

### Split the final grid into two vertical parts

#### `grid.split(<index of split>, axis=0)`

In [ ]:
grid = pgt.ModelGrid(grid.nodes_x, grid.nodes_y)
grid_bottom, grid_top = grid.split(14, axis=0)
grid_bottom.cell_mask = np.ma.masked_invalid(grid_bottom.xc).mask
grid_top.cell_mask = np.ma.masked_invalid(grid_top.xc).mask

fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid_bottom.plotCells(ax=ax, cell_kws=dict(facecolor=palette[2], alpha=0.75))
fig = grid_top.plotCells(ax=ax, cell_kws=dict(facecolor=palette[3], alpha=0.75))

### Splitting and linearly refining columns and rows
#### Split the final grid into two horizontal parts

`grid.split(<index of split>, axis=1)`

In [ ]:
grid = pgt.ModelGrid(grid.nodes_x, grid.nodes_y)
grid_left, grid_right = grid.split(8, axis=1)
grid_left.cell_mask = np.ma.masked_invalid(grid_left.xc).mask
grid_right.cell_mask = np.ma.masked_invalid(grid_right.xc).mask

fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid_left.plotCells(ax=ax, cell_kws=dict(facecolor=palette[4], alpha=0.75))
fig = grid_right.plotCells(ax=ax, cell_kws=dict(facecolor=palette[5], alpha=0.75))

#### Refine individual rows of the grid cells 
`grid.refine(<index of cell>, axis=0, n_points=<num. of divisions>)`

In [ ]:
grid = pgt.ModelGrid(grid.nodes_x, grid.nodes_y)
grid_refined = grid.refine(13, axis=0, n_points=2)
grid_refined.cell_mask = np.ma.masked_invalid(grid_refined.xc).mask

fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid_refined.plotCells(ax=ax, cell_kws=dict(facecolor=palette[0], alpha=0.75))

#### Refine individual columns of the grid cells 
`grid.refine(<index of cell>, axis=1, n_points=<num. of divisions>)`

In [ ]:
grid_refined_again = grid_refined.refine(10, axis=1, n_points=4)
grid_refined_again.cell_mask = np.ma.masked_invalid(grid_refined_again.xc).mask

fig, ax = plt.subplots(figsize=(7.5, 7.5))
fig = grid_refined_again.plotCells(ax=ax, cell_kws=dict(facecolor=palette[1], alpha=0.75))

### Chained operations
#### One big chained operation for fun

In [ ]:
grid1, grid2, grid3 = make_test_grids()
fig, ax = plt.subplots(figsize=(7.5, 7.5))

fig = (
  grid2.merge(grid3, how='vert', where='-', shift=2)
       .merge(grid1, how='horiz', where='-', shift=11) 
       .refine(10, axis=1, n_points=4)
       .refine(13, axis=0, n_points=2)
       .transform(lambda x: x*5 + 2)
       .update_cell_mask()
       .plotCells(ax=ax, cell_kws=dict(facecolor=palette[2], alpha=0.75))
)